In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import numpy as np
import joblib
import lightgbm as lgb

# Criar widgets para cada campo

# Tipo de emprego
job_label = widgets.Label("Tipo de Emprego:")
job_options = ["administrator", "blue-collar", "entrepreneur", "housemaid", "management", "retired", "self-employed", "services", "student", "technician", "unemployed"]
job_dropdown = widgets.Dropdown(options=job_options)

# Mês de contato
month_label = widgets.Label("Mês de Contato:")
month_options = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
month_dropdown = widgets.Dropdown(options=month_options)

# Escolaridade (Education)
education_label = widgets.Label("Escolaridade (Education):")
education_options = ["secondary", "tertiary", "primary", "unknown"]
education_dropdown = widgets.Dropdown(options=education_options)

# Idade
age_label = widgets.Label("Idade:")
age_entry = widgets.IntText()

# Salário (Balance)
balance_label = widgets.Label("Salário (Balance):")
balance_entry = widgets.FloatText()

# Duração do contato (Duration)
duration_label = widgets.Label("Duração do Contato (Duration):")
duration_entry = widgets.FloatText()

# Número de contatos realizados ao longo da campanha
campaign_label = widgets.Label("Número de Contatos (Campaign):")
campaign_entry = widgets.IntText()

# Dia do último contato
day_label = widgets.Label("Dia do Último Contato:")
day_entry = widgets.IntText()

# pdays
pdays_label = widgets.Label("Pdays:")
pdays_entry = widgets.IntText()

# previous
previous_label = widgets.Label("Previous:")
previous_entry = widgets.IntText()

# default
default_label = widgets.Label("Default (yes/no):")
default_dropdown = widgets.Dropdown(options=["yes", "no"])

# housing
housing_label = widgets.Label("Housing (yes/no):")
housing_dropdown = widgets.Dropdown(options=["yes", "no"])

# loan
loan_label = widgets.Label("Loan (yes/no):")
loan_dropdown = widgets.Dropdown(options=["yes", "no"])

# Função para submeter os dados e criar o DataFrame
def submit(_):
    education_map = {
        "primary": 0,
        "secondary": 0.5,
        "tertiary": 1,
        "unknown": np.nan
    }

    data = {
        "job": job_dropdown.value,
        "month": month_dropdown.value,
        "education": education_map[education_dropdown.value],
        "age": age_entry.value,
        "balance": balance_entry.value,
        "duration": duration_entry.value,
        "campaign": campaign_entry.value,
        "day": day_entry.value,
        "pdays": pdays_entry.value,
        "previous": previous_entry.value,
        "default": default_dropdown.value,
        "housing": housing_dropdown.value,
        "loan": loan_dropdown.value,
    }

    # Criação de colunas binárias para job
    job_dummies = {f"job_{job}": 1 if job == data["job"] else 0 for job in job_options}

    # Criação de colunas binárias para month
    month_dummies = {f"month_{month}": 1 if month == data["month"] else 0 for month in month_options}

    # Substituir "yes" e "no" por 1 e 0
    data["default"] = 1 if data["default"] == "yes" else 0
    data["housing"] = 1 if data["housing"] == "yes" else 0
    data["loan"] = 1 if data["loan"] == "yes" else 0

    # Combinar todos os dados em um único dicionário
    all_data = {**job_dummies, **month_dummies, **data}

    # Remover as chaves originais de 'job' e 'month' pois foram expandidas
    del all_data['job']
    del all_data['month']

    # Criar DataFrame
    df = pd.DataFrame([all_data])

    # Mostrar DataFrame
    display(df)

    # Normalização
    normalizando = ['education', 'age', 'balance', 'duration', 'campaign', 'day', 'pdays', 'previous']
    scaler = joblib.load('scaler.pkl')
    norm = scaler.transform(df[normalizando])  # Use transform instead of fit_transform

    # Convertendo o array numpy de volta para um DataFrame do Pandas
    df_norm = pd.DataFrame(norm, columns=normalizando)

    # Realizar a junção
    df_combined = df.copy()
    df_combined[normalizando] = df_norm[normalizando]

    # Carregar o modelo salvo
    loaded_model = joblib.load('best_model_lgbm.pkl')

    # Fazer previsões de probabilidade
    probabilidades = loaded_model.predict_proba(df_combined)

    # Determinar a classe prevista (0 ou 1) e a probabilidade associada
    resultados = []
    for prob in probabilidades:
        classe_predita = 1 if prob[1] >= 0.5 else 0  # Threshold de 0.5 para classificação binária
        probabilidade = prob[1] if classe_predita == 1 else prob[0]
        resultado = "Sucesso" if classe_predita == 1 else "Fracasso"
        resultados.append((resultado, probabilidade))

    # Imprimir os resultados
    for i, (resultado, probabilidade) in enumerate(resultados):
        print(f"Instância {i+1}: {resultado} com uma probabilidade de {probabilidade*100:.2f}%")

# Botão para submeter os dados
submit_button = widgets.Button(description="Submit")
submit_button.on_click(submit)

# Display widgets
display(job_label, job_dropdown,
        month_label, month_dropdown,
        education_label, education_dropdown,
        age_label, age_entry,
        balance_label, balance_entry,
        duration_label, duration_entry,
        campaign_label, campaign_entry,
        day_label, day_entry,
        pdays_label, pdays_entry,
        previous_label, previous_entry,
        default_label, default_dropdown,
        housing_label, housing_dropdown,
        loan_label, loan_dropdown,
        submit_button)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Label(value='Tipo de Emprego:')

Dropdown(options=('administrator', 'blue-collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self-…

Label(value='Mês de Contato:')

Dropdown(options=('jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'), value='…

...

Label(value='Loan (yes/no):')

Dropdown(options=('yes', 'no'), value='yes')

Button(description='Submit', style=ButtonStyle())

,job_administrator,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,age,balance,duration,campaign,day,pdays,previous,default,housing,loan
0,1,0,0,0,0,0,0,0,0,0,...,9,4.0,2.0,18,19,3,3,0,1,1


FileNotFoundError: [Errno 2] No such file or directory: 'scaler.pkl'